In [1]:
import pandas as pd
import numpy as np

# Opening

## CSV readers

In [2]:
df = pd.read_csv('reviews.csv', sep='|')
df_2 = pd.read_csv('reviews_75020_75007.csv', sep='|')

df = pd.concat([df, df_2])
df.reset_index(drop=True, inplace=True)
# df

# Cleaning

### Drop duplicates

In [3]:
# Drop duplicates except if comment value is null
df = df[(~df.duplicated()) | (df['comment'].isnull())]

### Check the number of empty fields

In [4]:
df.isnull().sum()
# Only the comments can be empty

name               0
address            0
comment         6003
comment_rate       0
dtype: int64

### Remove comments from pizzerias that are not in Paris

In [5]:
pd.set_option('display.max_rows', None)
# print(df[~df['address'].str.contains('Paris')]['address'].unique())

# We have to change an address that is in Paris. Others are out of Paris.
df.loc[ df['address'] == 'Angle rue de Boulanvilliers, 1 Rue des Bauches, 75016', 'address'] = 'Angle rue de Boulanvilliers, 1 Rue des Bauches, 75016 Paris'
# print(df[~df['address'].str.contains('Paris')]['address'].unique())
pd.set_option('display.max_rows', 10)

In [6]:
df = df[df['address'].str.contains('Paris', na=False)]
# print(df[~df['address'].str.contains('Paris')]['address'].unique())

### Remove newlines in the comments column

In [7]:
df = df.replace('\n',' ', regex=True)

### Add a postal_code column

In [8]:
df.address.str.split().str[-2]
df['postal_code'] = df.address.str.split().str[-2]

### Change comment_rate to int

In [9]:
df.comment_rate.str.split().str[0]
df['comment_rate'] = df.comment_rate.str.split().str[0]
df['comment_rate'] = pd.to_numeric(df['comment_rate'])

### Keep only traductions when there are some

In [12]:
# Remove all the parantheses, "Traduit par Google" and the spaces in the string
df['cleaned_comment'] = df['comment'].str.replace(r'[()]',"").str.replace("Traduit par Google","").str.strip()

# Split by Avis d'origine to separate the traduction and the original text
df['cleaned_comment'] = df['cleaned_comment'].str.split("Avis d'origine")

# Keep the first element and strip
df['cleaned_comment'] = df['cleaned_comment'].str[0].str.strip()

/tmp/ipykernel_7014/3744359659.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['cleaned_comment'] = df['comment'].str.replace(r'[()]',"").str.replace("Traduit par Google","").str.strip()


### Writing to CSV file

In [13]:
df.to_csv("comments_cleaned.csv", sep='|')